## 조금 에러 수정이 필요함. 그리고 모든 코드를 다 칠 필요는 없을 것 같음.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [2]:
# 데이터 증강 및 정규화를 위한 변환 설정
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # 데이터 증강: 랜덤하게 잘라내기
    transforms.RandomHorizontalFlip(),      # 데이터 증강: 랜덤하게 좌우 반전
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),  # 정규화
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# CIFAR10 데이터셋 불러오기
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

#위 불러오기 코드에서 train=True나 False는 어떤 역할을 하는지 퍼플렉시티한테 질문

train_ratio = 0.8  # 훈련 데이터의 비율

train_size = int(len(train_dataset) * train_ratio)
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# 데이터 로더 생성
batch_size = 128  # 원하는 배치 크기 설정
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#아래 코드는 아직 안 배웠음, 치는 거 넘어가기?
#입력 이미지 (3,32,32)
#출력 label -> 0~9

classes = 10  # CIFAR-10 클래스 수

# 모델 정의 (3채널 입력, CIFAR-10에 맞게 수정)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 합성곱 층
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # 3채널 입력
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        # 완전 연결 층
        self.fc1 = nn.Linear(64 * 16 * 16, 128)  # CIFAR-10은 32x32 이미지, MaxPool 후 16x16
        self.fc2 = nn.Linear(128, 10)  # CIFAR-10은 10개의 클래스
        # 활성화 함수와 드롭아웃
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.relu(self.conv1(x))   # [batch_size, 32, 32, 32]
        x = self.relu(self.conv2(x))   # [batch_size, 64, 32, 32]
        x = self.pool(x)               # [batch_size, 64, 16, 16]
        x = self.dropout(x)
        x = x.view(-1, 64 * 16 * 16)   # 평탄화
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
lr = 0.001
epochs = 50

model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [6]:
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

In [7]:
print("\033[34mtrain\033[0m")

train


In [8]:
def train(model, train_loader, criterion, optimizer, epoch):
    # 훈련 모드 설정
    model.train()

    train_loss=0
    train_correct=0
    train_total=0

    startStr =f'Epoch {epoch+1:2d}/{epochs} \033[34m' + 'Train'+ '\033[0m'
    with tqdm(total=len(train_loader), desc=startStr) as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            pbar.update(1)
        train_loss = train_loss / train_total
        train_accuracy = (train_correct / train_total)*100
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        endStr = f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}%'
        # tqdm동작이 다 끝나면 그때 아래의 코드가 동작
        print(endStr)
        #pbar.set_postfix(endStr)

In [9]:
def valid_or_test(mode, model, device, dataloader, epoch):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        if mode == 'valid':
            startStr = f'Epoch {epoch+1:2d}/{epochs} \033[34m' + 'Valid ' + '\033[0m'
        elif mode == 'test':
            startStr = f'Epoch {epoch+1:2d}/{epochs} \033[34m' + 'Test ' + '\033[0m'

        with tqdm(total=len(dataloader), desc=startStr) as pbar:
            for data, target in dataloader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                loss = criterion(outputs, target)

                loss += loss.item() * data.size(0)
                _, predicted = torch.max(outputs, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
                pbar.update(1)

            loss = loss / total
            accuracy = 100 * correct / total
            if mode == 'valid':
                endStr = f'Valid Loss: {loss:.4f}, Valid Acc: {accuracy:.3f}%'
            elif mode == 'test':
                endStr = f'Test Loss: {loss:.4f}, Test Acc: {accuracy:.3f}%'
            pbar.set_postfix_str(endStr)

    if mode == 'valid':
        valid_losses.append(loss)
        valid_accuracies.append(accuracy)
        print('-'*110)

In [10]:
for epoch in range(epochs):
    train(model, train_loader, criterion, optimizer, epoch)
    valid_or_test('valid', model, device, valid_loader, epoch)

valid_or_test('test', model, device, test_loader, epoch)

Epoch  1/50 Train: 100%|██████████| 313/313 [00:13<00:00, 23.83it/s]


Train Loss: 0.0132, Train Acc: 38.1700%


Epoch  1/50 Valid : 100%|██████████| 79/79 [00:10<00:00,  7.38it/s, Valid Loss: 0.0021, Valid Acc: 48.520%]


--------------------------------------------------------------------------------------------------------------


Epoch  2/50 Train: 100%|██████████| 313/313 [00:14<00:00, 21.53it/s]


...

Epoch 50/50 Train: 100%|██████████| 313/313 [00:12<00:00, 25.32it/s]


Train Loss: 0.0061, Train Acc: 72.3675%


Epoch 50/50 Valid : 100%|██████████| 79/79 [00:09<00:00,  8.43it/s, Valid Loss: 0.0008, Valid Acc: 74.080%]


--------------------------------------------------------------------------------------------------------------


Epoch 50/50 Test : 100%|██████████| 79/79 [00:09<00:00,  8.58it/s, Test Loss: 0.0009, Test Acc: 76.190%]


## 틀린 것이랑 맞은 것 사진 출력하는 것 프롬프트로 퍼플렉시티한테 질문

### pretrained 모델 사용한 코드

In [12]:
%pip install efficientnet_pytorch


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16458 sha256=cb4c79bf1f35a1a9600f21717e652af991d86e9d474dd4d3b8340bc8202e6e7c
  Stored in directory: c:\users\yjkim_pc\appdata\local\pip\cache\wheels\03\3f\e9\911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet

import time

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 증강 및 정규화 설정
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet 정규화 사용
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# CIFAR10 데이터셋 로드
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# 훈련/검증 데이터 분할
train_ratio = 0.8
train_size = int(len(train_dataset) * train_ratio)
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# 데이터 로더 생성
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# EfficientNet-B0 모델 로드 및 수정
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=10)  # CIFAR10 클래스 수: 10
model = model.to(device)

# 손실 함수, 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습 정보 저장 리스트
train_losses = []
valid_losses = []
train_accuracies = []
valid_accuracies = []

# 학습 함수 정의
def train(model, train_loader, criterion, optimizer, epoch, epochs):
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    start_str = f'Epoch {epoch+1:2d}/{epochs} \033[34mTrain\033[0m'
    with tqdm(total=len(train_loader), desc=start_str) as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            pbar.update(1)

        train_loss = train_loss / len(train_loader)
        train_accuracy = (train_correct / train_total) * 100
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        end_str = f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}%'
        pbar.set_postfix_str(end_str)

# 검증/테스트 함수 정의
def valid_or_test(mode, model, device, dataloader, epoch, epochs):
    model.eval()
    loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        if mode == 'valid':
            start_str = f'Epoch {epoch+1:2d}/{epochs} \033[34mValid\033[0m'
        elif mode == 'test':
            start_str = f'Epoch {epoch+1:2d}/{epochs} \033[34mTest\033[0m'

        with tqdm(total=len(dataloader), desc=start_str) as pbar:
            for data, target in dataloader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                loss = criterion(outputs, target)

                loss += loss.item() * data.size(0)
                _, predicted = torch.max(outputs, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
                pbar.update(1)

            loss = loss / total
            accuracy = 100 * correct / total
            if mode == 'valid':
                end_str = f'Valid Loss: {loss:.4f}, Valid Acc: {accuracy:.3f}%'
            elif mode == 'test':
                end_str = f'Test Loss: {loss:.4f}, Test Acc: {accuracy:.3f}%'
            pbar.set_postfix_str(end_str)

    if mode == 'valid':
        valid_losses.append(loss)
        valid_accuracies.append(accuracy)
        print('-' * 110)

# 학습 시작
epochs = 7
for epoch in range(epochs):
    train(model, train_loader, criterion, optimizer, epoch, epochs)
    valid_or_test('valid', model, device, valid_loader, epoch, epochs)

# 테스트
valid_or_test('test', model, device, test_loader, epoch, epochs)

Files already downloaded and verified
Files already downloaded and verified


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to C:\Users\YJKIM_PC/.cache\torch\hub\checkpoints\efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:01<00:00, 11.8MB/s]


Loaded pretrained weights for efficientnet-b0


Epoch  1/7 Valid: 100%|██████████| 79/79 [00:10<00:00,  7.49it/s, Valid Loss: 0.0016, Valid Acc: 69.850%]


--------------------------------------------------------------------------------------------------------------


...

Epoch  6/7 Valid: 100%|██████████| 79/79 [00:11<00:00,  6.97it/s, Valid Loss: 0.0008, Valid Acc: 81.420%]


--------------------------------------------------------------------------------------------------------------


Epoch  7/7 Valid: 100%|██████████| 79/79 [00:11<00:00,  7.04it/s, Valid Loss: 0.0014, Valid Acc: 82.500%]


--------------------------------------------------------------------------------------------------------------


Epoch  7/7 Test: 100%|██████████| 79/79 [00:09<00:00,  7.92it/s, Test Loss: 0.0016, Test Acc: 83.190%]
